In [1]:
# !pip install pytorch
# !pip install transformers
#! sudo apt install tesseract-ocr
#! pip install pytesseract
#! pip install transformers datasets

In [1]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix,ConfusionMatrixDisplay
from sklearn.preprocessing import LabelEncoder

import os
import numpy as np
np.random.seed(42)
import pandas as pd
import tensorflow as tf

# from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.layers import Layer#, InputSpec
from tensorflow.keras import initializers, regularizers, constraints
#from tensorflow.keras.callbacks import Callback
from tensorflow.keras.layers import Input, Dense, GRU, Bidirectional, TimeDistributed, Dropout
from tensorflow.keras import backend as K
from tensorflow.keras.models import Model

import warnings
warnings.filterwarnings('ignore')

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

2022-04-04 14:10:24.825518: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-04-04 14:10:24.825686: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
#label encoding
train_text="/media/umar_visionx/Backup Plus/Active/Faizan/textrect_dataset"
labl=[]
for folder in os.listdir(train_text):
  labl.append(folder)#making the list of folder names which are labels


#encoding the labels from the list
label_index={}
for index,label in enumerate(labl):
  label_index[label]=index

label_index#label to index

{'Admin Note': 0,
 'Clinical History And Summary': 1,
 'Consult Note': 2,
 'Cover Page': 3,
 'Imaging Note': 4,
 'Insurance Authorization': 5,
 'Intake Forms': 6,
 'Lab Test': 7,
 'Other': 8,
 'Patient Profile': 9,
 'Prescriptions': 10,
 'Referral Letter': 11,
 'Requisition Form': 12}

In [3]:
#making class weights dictionary (our classes are imbalance)
#getting class having maximum samples
class_samples={}
for folder in os.listdir(train_text):
    folder_path=os.path.join(train_text,folder)
    class_samples[folder]=len(os.listdir(folder_path))
print(f'max samples in class ( {max(class_samples, key=class_samples.get)}={class_samples[max(class_samples, key=class_samples.get)]})')
max_samples=int(class_samples[max(class_samples, key=class_samples.get)])
print('***********')
#getting class weights
weights={}
for folder in os.listdir(train_text):
    folder_path=os.path.join(train_text,folder)
    weights[folder]=round(max_samples/int(len(os.listdir(folder_path))),2)
print(f'class weights: {weights}')
print("********")
#changing the names of keys in class weight dictionary
class_weight={}
for (keys1,values1),(keys2,values2) in zip(weights.items(),label_index.items()):

    class_weight[int(values2)]=values1

print(f'class/label weights: {class_weight}')

max samples in class ( Referral Letter=2363)
***********
class weights: {'Admin Note': 4.02, 'Clinical History And Summary': 2.26, 'Consult Note': 1.64, 'Cover Page': 1.53, 'Imaging Note': 4.81, 'Insurance Authorization': 2.44, 'Intake Forms': 102.74, 'Lab Test': 3.03, 'Other': 2.41, 'Patient Profile': 32.82, 'Prescriptions': 42.96, 'Referral Letter': 1.0, 'Requisition Form': 1.8}
********
class/label weights: {0: 4.02, 1: 2.26, 2: 1.64, 3: 1.53, 4: 4.81, 5: 2.44, 6: 102.74, 7: 3.03, 8: 2.41, 9: 32.82, 10: 42.96, 11: 1.0, 12: 1.8}


In [4]:
saved_data='/media/umar_visionx/Backup Plus/Active/Faizan/textrect_saved_vectors'

In [5]:
import pickle
#loading data from pickle file
def ext_data(filepath):

    data = []
    with open(filepath, 'rb') as fr:
        try:
            while True:
                data.append(pickle.load(fr))
        except EOFError:
            pass
    return data

In [6]:
from tqdm import tqdm
train_feat_path='/media/umar_visionx/Backup Plus/Active/Faizan/saved_vectors/train_feat.pickle'
train_label_path='/media/umar_visionx/Backup Plus/Active/Faizan/saved_vectors/train_label.pickle'
test_feat_path='/media/umar_visionx/Backup Plus/Active/Faizan/saved_vectors/test_feat.pickle'
test_label_path='/media/umar_visionx/Backup Plus/Active/Faizan/saved_vectors/test_label.pickle'
doc_vectors_list_arr=ext_data(train_feat_path)
doc_label_list_arr=ext_data(train_label_path)
test_vector_list_arr=ext_data(test_feat_path)
test_label_list_arr=ext_data(test_label_path)

In [7]:
# del doc_vectors_list_arr
# del doc_label_list_arr

In [8]:
#this list have document vectors,every vector,every document have 512 token vectors and every oken vector have 3072 entries
print(len(doc_vectors_list_arr))
print(len(doc_label_list_arr))
print(len(doc_vectors_list_arr[1]))
print(len(doc_label_list_arr))
print(len(doc_vectors_list_arr[1][1]))

11660
11660
512
11660
768


In [9]:
#splitiing the data into train and validation
x_t, x_v, y_t, y_v = train_test_split(doc_vectors_list_arr, doc_label_list_arr, test_size=0.1,random_state=42,stratify=doc_label_list_arr)
print(len(x_t))
print(len(x_v))

10494
1166


In [10]:
#converting features into numpy array
x_train=[]
y_train=[]
x_val=[]
y_val=[]
x_test=[]
y_test=[]

# #train set
for ent in x_t:
    x_train.append(np.array(ent,dtype=np.float32))

for ent in y_t:
    y_train.append(np.array(ent,dtype=np.int32))

    #validation set
for ent in x_v:
    x_val.append(np.array(ent,dtype=np.float32))

for ent in y_v:
     y_val.append(np.array(ent,dtype=np.int32))

#test set
for ent in test_vector_list_arr:
    x_test.append(np.array(ent,dtype=np.float32))

for ent in test_label_list_arr:
    y_test.append(np.array(ent,dtype=np.int32))

print(len(x_train))
print(len(y_train))
print(len(x_val))
print(len(y_val))
print(len(x_test))
print(len(y_test))

    


10494
10494
1166
1166
1810
1810


In [11]:
# del x_train
# del y_train
# del x_val
# del y_val

In [12]:
# pip install -U gast==0.2.2

In [13]:
class AttentionLayer(Layer):
    """
    Hierarchial Attention Layer as described by Hierarchical Attention Networks for Document Classification(2016)
    - Yang et. al.
    Source: https://www.cs.cmu.edu/~hovy/papers/16HLT-hierarchical-attention-networks.pdf
    Theano backend
    """


    def __init__(self,attention_dim=768,return_coefficients=False,**kwargs):
        # Initializer
        self.init = initializers.get('glorot_uniform') # initializes values with uniform distribution
        self.return_coefficients = return_coefficients
        self.attention_dim = attention_dim
        self.supports_masking = True
        super(AttentionLayer, self).__init__(**kwargs)

    def build(self, input_shape):
        # print(input_shape[0])
        # print(input_shape[-1])
        # print(input_shape[1])
        # Builds all weights
        # u = context vector,W = Weight matrix, b = bias vector
        assert len(input_shape) == 3
        self.u = K.variable(self.init((self.attention_dim, 1)),name='u')
        self.b = K.variable(self.init((self.attention_dim, )),name='b')
        self.W = K.variable(self.init((int(input_shape[-1]), self.attention_dim)),name='W')
        
        self._trainable_weights = [self.W, self.b, self.u]

        super(AttentionLayer, self).build(input_shape)

    def compute_mask(self, input, input_mask=None):
        return None

    def call(self, hit, mask=None):
        # Here, the actual calculation is done
        uit = K.bias_add(K.dot(hit, self.W),self.b)
        uit = K.tanh(uit)
        
        ait = K.dot(uit, self.u)
        ait = K.squeeze(ait, -1)
        ait = K.exp(ait)
        
        if mask is not None:
            ait *= K.cast(mask, K.floatx())

        ait /= K.cast(K.sum(ait, axis=1, keepdims=True) + K.epsilon(), K.floatx())
        ait = K.expand_dims(ait)
        weighted_input = hit * ait
        
        if self.return_coefficients:
            return [K.sum(weighted_input, axis=1), ait]
        else:
            return K.sum(weighted_input, axis=1)

    def compute_output_shape(self, input_shape):
        if self.return_coefficients:
            return [(input_shape[0], int(input_shape[-1])), (input_shape[0], int(input_shape[-1]), 1)]
        else:
            return input_shape[0], int(input_shape[-1])

    # def get_config(self):

    #     config = super().get_config().copy()
    #     config.update({
    #         'attention_dim ': self.attention_dim ,
    #         'return_coefficients': self.return_coefficients,
    #     })
    #     return config

    def get_config(self):#to avoiding saving problem.check(https://stackoverflow.com/questions/50837728/valueerror-unknown-layer-capsulelayer and    
        #https://stackoverflow.com/questions/58678836/notimplementederror-layers-with-arguments-in-init-must-override-get-conf)
        # For serialization with 'custom_objects'
        config = super().get_config()
        config['attention_dim'] = self.attention_dim
        config['return_coefficients'] = self.return_coefficients
        return config

In [ ]:
import tensorflow
import tensorflow as tf
# embedding_matrix = np.random.random((VOCAB_SIZE, EMBEDDING_DIM))
# max_sentenses=1
# embedding_layer = Embedding(VOCAB_SIZE,EMBEDDING_DIM,input_length=MAX_SEQUENCE_LENGTH,trainable=False,weights=[embedding_matrix],name='word_embedding')
MAX_SEQUENCE_LENGTH=512
EMBEDDING_DIM=768
# max_sentenses=1

# Words  attention
emb_input = Input(shape=(MAX_SEQUENCE_LENGTH,EMBEDDING_DIM), dtype='float32',name='embeddings_input')
#word_sequences = embedding_layer(word_input)
gru_word = Bidirectional(GRU(768, return_sequences=True),name='word_GRU')(emb_input)#by default if we set treturn_sequence=False it will
#return only last words hidden state is returm only as the output having vector dimensions=2*GRU units 
#for bidirectional
# word_dense = Dense(1536, activation='relu', name='word_dense')(gru_out) 
word_attention,word_coff= AttentionLayer(EMBEDDING_DIM,return_coefficients=True,name='word_attention')(gru_word)

dense_out = Dense(512,activation='relu', name='word_dense')(word_attention)#we may use different number of units
dropout = Dropout(0.5,name='dropout')(dense_out)
preds = Dense(13, activation='softmax',name='output')(dropout)
model = Model(emb_input, preds)




In [29]:

model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['acc'])
print(model.summary())


Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embeddings_input (InputLaye  [(None, 512, 768)]       0         
 r)                                                              
                                                                 
 word_GRU (Bidirectional)    (None, 512, 1536)         7087104   
                                                                 
 word_attention (AttentionLa  [(None, 1536),           1181184   
 yer)                         (None, 512, 1)]                    
                                                                 
 word_dense (Dense)          (None, 512)               786944    
                                                                 
 dropout (Dropout)           (None, 512)               0         
                                                                 
 output (Dense)              (None, 13)                6669

In [14]:
#data loader
from tensorflow.keras.utils import Sequence
import math
class DataGenerator(Sequence):

    def __init__(self, x_train, y_train,batch_size):
        self.x, self.y = x_train, y_train
        self.batch_size = batch_size

    def __len__(self):
        return math.ceil(len(self.x) / self.batch_size)#ValueError: axes don't match array

    def __getitem__(self, idx):
        # batch_x = self.x[idx * self.batch_size:(idx + 1) *
        # self.batch_size]
        # batch_y = self.y[idx * self.batch_size:(idx + 1) *
        # self.batch_size]
        x_tr = np.array(self.x[idx * self.batch_size:(idx + 1) *
        self.batch_size])#we also added an extra dimension according to our requirement
        y_tr = np.array(self.y[idx * self.batch_size:(idx + 1) *
        self.batch_size])


        return x_tr,y_tr

In [15]:
train_generator=DataGenerator(x_train, y_train,32)
val_generator=DataGenerator(x_val, y_val,32)
#test_generator = DataGenerator(x_test,y_test,32)

In [16]:

x_tst=np.array(x_test)
y_ts=np.array(y_test)

In [17]:
#checking the total samples in each class(label)
from collections import Counter
print(f"labels: {Counter(doc_label_list_arr).keys()}")
print(f"samples in each label: {Counter(doc_label_list_arr).values()}")


labels: dict_keys([5, 2, 1, 7, 0, 11, 8, 3, 4, 12, 10, 9, 6])
samples in each label: dict_values([969, 1438, 1045, 780, 588, 2363, 982, 1542, 491, 1312, 55, 72, 23])


In [24]:
#generator=training_generator
#from tensorflow.keras.callbacks import EarlyStopping,ModelCheckpoint
#batch_size = 32
checkpoint_path='/media/umar_visionx/Backup Plus/Active/Faizan/word_att_model'
#es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=5)#early stopping
#mc = ModelCheckpoint(checkpoint_path, monitor='val_acc', mode='max', verbose=1, save_best_only=True)#model checkpoint
loss_list=[]
#loss_list.append(history.history['val_loss'][0])
history=model.fit(train_generator,validation_data=val_generator, epochs=1,class_weight=class_weight,shuffle=True)
loss_list.append(history.history['val_loss'][0])
for i in range(20):
    history=model.fit(train_generator,validation_data=val_generator, epochs=1,class_weight=class_weight,shuffle=True)
    loss=history.history['val_loss'][0]
    if loss<loss_list[-1]:
        #saving the model
        model.save(checkpoint_path)
        loss_list.append(history.history['val_loss'][0])

2022-04-04 10:20:41.880122: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 50331648 exceeds 10% of free system memory.
2022-04-04 10:20:51.285443: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 50331648 exceeds 10% of free system memory.
2022-04-04 10:20:51.360412: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 50331648 exceeds 10% of free system memory.
2022-04-04 10:20:51.460537: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 50331648 exceeds 10% of free system memory.
2022-04-04 10:20:51.537034: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 50331648 exceeds 10% of free system memory.


328/328 [==============================] - 1522s 5s/step - loss: 0.6148 - acc: 0.8902 - val_loss: 0.4619 - val_acc: 0.8799


2022-04-04 11:36:28.174565: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: /media/umar_visionx/Backup Plus/Active/Faizan/word_att_model/assets


INFO:tensorflow:Assets written to: /media/umar_visionx/Backup Plus/Active/Faizan/word_att_model/assets


290/328 [=========================>....] - ETA: 2:49 - loss: 0.9638 - acc: 0.8411

In [35]:
# #saving the model
# model.save('/media/umar_visionx/Backup Plus/Active/Faizan/word_att_model')

INFO:tensorflow:Assets written to: /media/umar_visionx/Backup Plus/Active/Faizan/word_att_model/assets


INFO:tensorflow:Assets written to: /media/umar_visionx/Backup Plus/Active/Faizan/word_att_model/assets


In [18]:
model = tf.keras.models.load_model('/media/umar_visionx/Backup Plus/Active/Faizan/word_att_model')
model.summary()

2022-04-04 14:20:00.155311: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-04-04 14:20:00.179663: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory
2022-04-04 14:20:00.183929: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: libcublasLt.so.11: cannot open shared object file: No such file or directory
2022-04-04 14:20:00.254583: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcufft.so.10'; dlerror: libcufft.so.10: cannot open shared object file: No such file or directory
2022-04-04 14:20:00.272795: W tensorflow/stream_executor/platform/default/dso_loader.cc:64

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embeddings_input (InputLaye  [(None, 512, 768)]       0         
 r)                                                              
                                                                 
 word_GRU (Bidirectional)    (None, 512, 1536)         7087104   
                                                                 
 word_attention (AttentionLa  [(None, 1536),           1181184   
 yer)                         (None, 512, 1)]                    
                                                                 
 word_dense (Dense)          (None, 512)               786944    
                                                                 
 dropout (Dropout)           (None, 512)               0         
                                                                 
 output (Dense)              (None, 13)                6669

In [34]:
history.history

{'loss': [0.09565596282482147],
 'acc': [0.9690299034118652],
 'val_loss': [0.44553402066230774],
 'val_acc': [0.8936535120010376]}

In [19]:
score = model.evaluate(x_tst, y_ts, verbose = 1) 

print('Test loss:', score[0]) 
print('Test accuracy:', score[1])

2022-04-04 14:20:13.932218: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 2846883840 exceeds 10% of free system memory.
2022-04-04 14:20:18.836347: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 50331648 exceeds 10% of free system memory.
2022-04-04 14:20:18.836425: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 50331648 exceeds 10% of free system memory.
2022-04-04 14:20:18.836699: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 50331648 exceeds 10% of free system memory.
2022-04-04 14:20:18.836791: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 50331648 exceeds 10% of free system memory.


57/57 [==============================] - 88s 1s/step - loss: 1.6216 - acc: 0.6845
Test loss: 1.6215522289276123
Test accuracy: 0.6845303773880005


In [20]:
predictions=model.predict(x_tst)

In [21]:
y_ts[1:5]

array([0, 0, 0, 0], dtype=int32)

In [22]:
predictions[1:5]

array([[9.99852657e-01, 3.51574217e-20, 6.65618336e-06, 1.42585204e-05,
        4.12751638e-16, 1.62662474e-07, 0.00000000e+00, 4.11187777e-17,
        4.24932978e-08, 1.11167533e-20, 1.03712668e-31, 1.07694934e-04,
        1.85271183e-05],
       [9.76409435e-01, 2.12820221e-17, 9.37808945e-05, 4.39146424e-06,
        4.55390066e-12, 1.04569726e-05, 0.00000000e+00, 1.56020121e-13,
        3.63488617e-08, 3.30945011e-19, 3.74633896e-26, 2.04123762e-02,
        3.06940940e-03],
       [9.99008596e-01, 5.26150362e-20, 1.60840250e-06, 9.80917597e-04,
        2.54986934e-16, 5.32482645e-08, 0.00000000e+00, 3.04127195e-19,
        1.02670685e-06, 7.93316234e-24, 8.28183892e-34, 7.74346336e-06,
        1.47977097e-08],
       [9.97114301e-01, 3.89703236e-16, 2.80252698e-05, 2.78629502e-03,
        3.31891876e-13, 1.41173791e-06, 5.19026019e-37, 1.37459450e-15,
        1.58872790e-05, 8.65341685e-20, 6.27584032e-27, 5.37528940e-05,
        2.60057476e-07]], dtype=float32)

In [23]:

pred = np.argmax(predictions, axis = 1)

print(pred) 
#print(label)

[ 0  0  0 ... 12 12 12]


In [24]:
from sklearn.metrics import classification_report
print(classification_report(y_ts,pred))

              precision    recall  f1-score   support

           0       0.58      0.64      0.61        83
           1       0.60      0.69      0.64       336
           2       0.57      0.72      0.64       369
           3       0.92      0.93      0.93       179
           4       0.72      0.60      0.66        63
           5       0.36      0.26      0.30        31
           6       0.00      0.00      0.00         8
           7       0.89      0.83      0.86       181
           8       0.76      0.32      0.45       266
           9       0.63      0.76      0.69        41
          10       1.00      0.22      0.36         9
          11       0.68      0.89      0.77       169
          12       0.93      0.75      0.83        75

    accuracy                           0.68      1810
   macro avg       0.67      0.59      0.60      1810
weighted avg       0.70      0.68      0.67      1810



In [25]:
from sklearn.metrics import accuracy_score
print(accuracy_score(y_test,pred))

0.6845303867403315
